Testing a preliminary HDPE, LDPE & PP dataset

In [149]:
# Step 1 Load the Data
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the data from the concatenated CSV file
data = pd.read_csv("allTogether.csv")

In [150]:
#Step 2: Prepare the Data
# Extract features (IR wavelength values) and target (plastic type labels)
features = data.drop("Plastic_Type", axis=1)
labels = data["Plastic_Type"]

# Handle negative values (e.g., by normalization)
features = (features - features.min()) / (features.max() - features.min())


In [151]:
# Step 3: Encode the Target Labels
# Use Label Encoding to convert target labels into numerical values
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Encode the target labels into one-hot encoded format
encoded_labels = tf.keras.utils.to_categorical(encoded_labels, num_classes=4)  # 4 classes (3 plastics + "other")


In [152]:
# Step 4: Split the Data into Training and Testing Sets
# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

In [153]:
# Step 5: Build the TensorFlow Model
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(8,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # 4 output units for 3 plastic types and "other"
])

# Compile the model with categorical_crossentropy loss
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)


In [154]:
# Step 6: Train the Model
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/200
3/3 [==============================] - 1s 118ms/step - loss: 1.4683 - accuracy: 0.2208 - val_loss: 1.4054 - val_accuracy: 0.3333
Epoch 2/200
3/3 [==============================] - 0s 20ms/step - loss: 1.4097 - accuracy: 0.3247 - val_loss: 1.3620 - val_accuracy: 0.3333
Epoch 3/200
3/3 [==============================] - 0s 19ms/step - loss: 1.3649 - accuracy: 0.3247 - val_loss: 1.3313 - val_accuracy: 0.4444
Epoch 4/200
3/3 [==============================] - 0s 26ms/step - loss: 1.3328 - accuracy: 0.4935 - val_loss: 1.3063 - val_accuracy: 0.7778
Epoch 5/200
3/3 [==============================] - 0s 20ms/step - loss: 1.3051 - accuracy: 0.5844 - val_loss: 1.2848 - val_accuracy: 0.7778
Epoch 6/200
3/3 [==============================] - 0s 23ms/step - loss: 1.2815 - accuracy: 0.5714 - val_loss: 1.2655 - val_accuracy: 0.7778
Epoch 7/200
3/3 [==============================] - 0s 23ms/step - loss: 1.2592 - accuracy: 0.5844 - val_loss: 1.2473 - val_accuracy: 0.7778
Epoch 8/200
3/3 [==

In [155]:
# Step 7: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

1/1 [==============================] - 0s 34ms/step - loss: 0.4940 - accuracy: 0.7727
Test Accuracy: 0.7727


In [156]:
# Step 8: Make predictions
new_data = pd.DataFrame({
    "IR_Wavelength_1": [761.33],
    "IR_Wavelength_2": [3897.67],
    "IR_Wavelength_3": [8964],
    "IR_Wavelength_4": [11729.33],
    "IR_Wavelength_5": [11358.67],
    "IR_Wavelength_6": [8942.33],
    "IR_Wavelength_7": [4617.33],
    "IR_Wavelength_8": [3227]
})

# Preprocess the new data (e.g., normalization) and predict the plastic type probabilities
new_data_normalized = (new_data - features.min()) / (features.max() - features.min())
predictions = model.predict(new_data_normalized)

# Decode the predictions to get the plastic type probabilities
plastic_type_probabilities = predictions[0][:3]  # First three elements are the probabilities for HDPE, LDPE, and PE
other_probability = predictions[0][3]  # Fourth element is the probability for "other"

print("Plastic Type Probabilities:")
print("HDPE:", plastic_type_probabilities[0])
print("LDPE:", plastic_type_probabilities[1])
print("PP:", plastic_type_probabilities[2])
print("Other:", other_probability)

1/1 [==============================] - 0s 66ms/step
Plastic Type Probabilities:
HDPE: 0.0
LDPE: 0.0
PP: 1.0
Other: 0.0
